In [448]:
import plotly.express as px
import json 
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import numpy as np

#abriendo el archivo json
with open ('datos.json','r')as gr:
    dt=gr.read()
granma=json.loads(dt)



df=pd.DataFrame(dict(Mujeres=granma['municipios']['RioCauto']['residentesH_M_14-22'][1],años=[x for x in granma['años']if x>=2014]))
df1=pd.DataFrame(dict(Hombres=granma['municipios']['RioCauto']['residentesH_M_14-22'][1],años=[x for x in granma['años']if x>=2014]))


fig=px.line_polar(df,r='años',theta='Mujeres',text="años")
fig.update_traces(fill='toself',textposition='top center')
fig1=px.bar(df1,x='años',y='Hombres', color='Hombres')

fig.show()
fig1.show()





In [471]:

#función para sacar valores de los residentes H_M por muicipios del json
def llenar_lista()->list:
    lista=[]
    for keys,values in granma.items():
        if keys=="municipios": #accedo a la llave municipios que es un diccionario
            for clave,valor in values.items(): 
                if clave in granma["listamunis"]:
                     for key,value in valor.items():
                            if key =='residentesH_M_14-22':
                                lista.extend(value)
              
    return lista


def agregar_mun(lista):
    mun=granma['listamunis']
    for i in range(len(mun)):
            lista[i].insert(0,mun[i])
  
    

#lista con todos los valores de los residentes 14-2022  
lista=llenar_lista()



#lista de residentes hombres14-2022
listaH=[y for x,y in enumerate(lista) if x%2 == 0 ]

#función para que le agrega la el municipio a la lista de listas
#agregar_mun(listaH)


#lista de residentes mujeres 14-2022
listaM=[y for x,y in enumerate(lista)if x%2 != 0]


#agregar_mun(listaM)

#data frame de la lista de mujeres
dfM=pd.DataFrame(listaM,columns= [x for x in granma['años']if x>=2014])



# matriz de correlación del data frame de la lista de mujeres


#data frame de la lista de hombres
dfH=pd.DataFrame(listaH,columns=[x for x in granma['años']if x>=2014])

#grafico de calor de la población Mujer 2014-22
fig=px.imshow(dfM,x=[x for x in granma['años']if x>=2014],y=granma['listamunis'])
fig.show()

#grafico de calor de la población hombre 2014-22
fig=px.imshow(dfH,x=[x for x in granma['años']if x>=2014],y=granma['listamunis'])
fig.show()